# Load Tensorflow and check GPU availability

In [1]:
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from pandas import DataFrame
from pandas.core.dtypes import common as com
from tensorflow.python.client import device_lib

for device in device_lib.list_local_devices():
    print(device.physical_device_desc)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# TODO: Zkusit 3 stupně na NN, SVM a porovnat s LightGBM + XGBoost

# 4 x Malware
# 4x  Phishing
# + DGA (jeden stupen)

2025-05-01 17:06:06.354553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-01 17:06:06.354586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-01 17:06:06.355921: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-01 17:06:06.362648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-01 17:06:06.970077: W tensorflow/compiler/tf2


device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


2025-05-01 17:06:08.692794: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-01 17:06:08.730954: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-01 17:06:08.735072: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
from core.loader import Loader, DataMannager

# Load input datasets

In [3]:


# #############################################################
# EDIT this to specify benign / malicious datasets to use     #
# #############################################################
benign_dataset_filenames = [
    'parkets/benign/benign_2312_anonymized_HTML.parquet', 
    'parkets/benign/umbrella_benign_FINISHED_HTML.parquet'
        
]
malicious_dataset_filenames = [
    'parkets/phishing_2406_strict_HTML.parquet'
]

# lex-dga-830k-pick.parquet LABEL je jmeno rodiny
#  'parkets/malware_2406_strict_HTML.parquet'


# #############################################################
# EDIT this for to set appropriate labels (malware, dga, ...) #
# #############################################################
benign_label = "benign"
malicious_label = "phishing"
# #############################################################

class_map = {benign_label: 0, malicious_label: 1}
# print labels from malicious datasets

loader = Loader(benign_dataset_filenames, malicious_dataset_filenames, benign_label=benign_label, malicious_label=malicious_label, subsample=1.0)

# dump columns names to text file
dm = DataMannager(loader, class_map)


### Generate subsets of the training dataset

In [6]:
from core.loader import DataMannager

dm.load()

aggregates = [
            ["lex_"],
            ["lex_", "dns_", "ip_", "geo_"],
            ["lex_", "dns_", "ip_", "tls_", "geo_", "rdap_"],
        ]

dm.segment(aggregates)





Segmenter initialized. Creating subsets...


/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

{'lex_agg':            label  lex_name_len  lex_has_digit  lex_phishing_keyword_count  \
 0         benign            23              1                           0   
 1         benign            24              1                           0   
 2         benign             8              1                           0   
 3         benign            26              1                           0   
 4         benign            26              1                           0   
 ...          ...           ...            ...                         ...   
 980744  phishing            12              1                           0   
 980745  phishing            43              1                           0   
 980746  phishing            21              1                           0   
 980747  phishing            41              0                           1   
 980748  phishing            79              1                           0   
 
         lex_benign_keyword_count  lex_consecutive_

In [7]:

dm.save_data()

Saved validation stage 1 to ./data/whole_split_1_phishing.pkl
Saved validation stage 2 to ./data/whole_split_2_phishing.pkl
Saved validation stage 3 to ./data/whole_split_3_phishing.pkl
